<a href="https://colab.research.google.com/github/PikaHackdon/AI-Certification/blob/master/VITS_Paimon_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **派蒙语音合成**

![paimon](https://raw.githubusercontent.com/JOETtheIV/vits-mandarin-biaobei/main/IMG_9368.PNG)

作者：VentiJ

邮箱：joeyventicup@gmail.com

学校：WHU

QQ：2997685348

B站：https://space.bilibili.com/327540506/video

项目代码：https://github.com/JOETtheIV/vits-mandarin-biaobei 

# 配置环境

请先点击右上角的**连接**，获取GPU运算资源。

点击更改运行时类型（change runtime type）
将硬件加速器改为GPU


然后运行该代码块。可能需要几分钟……

In [ ]:
!git clone https://github.com/JOETtheIV/vits-mandarin-biaobei
%cd /content/vits-mandarin-biaobei

!pip install -r requirements.txt
!sudo apt-get install espeak -y

%cd /content/vits-mandarin-biaobei/monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir results

Cloning into 'vits-mandarin-biaobei'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 135 (delta 47), reused 38 (delta 16), pack-reused 33
Receiving objects: 100% (135/135), 3.39 MiB | 12.32 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/vits-mandarin-biaobei
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 8.0 MB/s 
     |████████████████████████████████| 183 kB 70.4 MB/s 
     |████████████████████████████████| 11.6 MB 48.3 MB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
     |████████████████████████████████| 25.9 MB 1.4 MB/s 
     |████████████████████████████████| 238 kB 72.7 MB/s 
     |████████████████████████████████| 1.4 MB 62.0 MB/s 
     |████████████████████████████████| 56 kB 5.5 MB/s 
     |████████████████████████████████| 197 kB 68.2 MB/s 
     

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 

In [ ]:
import os
path = '/content/vits-mandarin-biaobei'
os.chdir(path)
print(os.getcwd())

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

/content/vits-mandarin-biaobei


# 加载模型 (1200 epochs)







In [ ]:
!gdown --id '1Gw-eNjB91MnnlZ1KTGNANRy4wbXTxN4B' --output G_paimon_tmp.pth

hps = utils.get_hparams_from_file("./configs/biaobei_base.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint('/content/vits-mandarin-biaobei/G_paimon_tmp.pth', net_g, None)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Gw-eNjB91MnnlZ1KTGNANRy4wbXTxN4B
To: /content/vits-mandarin-biaobei/G_paimon_tmp.pth
100% 437M/437M [00:05<00:00, 76.1MB/s]


# 语音合成



你还可以通过调节`length_scale`来控制说话的速度！注意`length_scale`默认值为1.0，值越大说话越**慢**。

**Tips**

1. 测试发现太长或太短的句子效果都不太好，参考文本建议分句使用。
2. 「、」的停顿效果不是很理想，建议使用「。」或「…」。
3. 由于数据集中去掉了大部分H片段，余下数据里各个人物说话的语调都比较平稳。因此使用「！」可能会出现破音。

为生合成的语音设定一个文件名。注意不需要加扩展名！

命名后运行该代码块，你将在左侧文件系统中/content/vits-mandarin-biaobei/test.wav找到它！

In [1]:
import soundfile as sf
text = "\u4E0B\u9762\u7ED9\u5927\u5BB6\u7B80\u5355\u4ECB\u7ECD\u4E00\u4E0B\u600E\u4E48\u4F7F\u7528\u8FD9\u4E2A\u6559\u7A0B\u5427\uFF01\u9996\u5148\u6211\u4EEC\u8981\u6709\u9B54\u6CD5\uFF0C\u624D\u80FD\u8BBF\u95EE\u5230\u8C37\u6B4C\u7684\u4E91\u5E73\u53F0\u3002\u70B9\u51FB\u8FDE\u63A5\u5E76\u66F4\u6539\u8FD0\u884C\u65F6\u7C7B\u578B\uFF0C\u8BBE\u7F6E\u786C\u4EF6\u52A0\u901F\u5668\u4E3AGPU\u3002\u7136\u540E\uFF0C\u6211\u4EEC\u518D\u4ECE\u5934\u5230\u5C3E\u6328\u4E2A\u70B9\u51FB\u6BCF\u4E2A\u4EE3\u7801\u5757\u7684\u8FD0\u884C\u6807\u5FD7\u3002\u53EF\u80FD\u9700\u8981\u7B49\u5F85\u4E00\u5B9A\u7684\u65F6\u95F4\u3002\u5F53\u6211\u4EEC\u8FDB\u884C\u5230\u8BED\u97F3\u5408\u6210\u90E8\u5206\u65F6\uFF0C\u5C31\u53EF\u4EE5\u66F4\u6539\u8981\u8BF4\u7684\u6587\u672C\uFF0C\u5E76\u8BBE\u7F6E\u4FDD\u5B58\u7684\u6587\u4EF6\u540D\u5566\u3002" #@param {type: 'string'}
length_scale = 1 #@param {type:"slider", min:0.1, max:3, step:0.05}
filename = 'test' #@param {type: "string"}
audio_path = f'/content/vits-mandarin-biaobei/{filename}.wav'
stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
sf.write(audio_path,audio,samplerate=hps.data.sampling_rate)

NameError: ignored

# 参考

https://github.com/jaywalnut310/vits

https://colab.research.google.com/drive/1eFmnzUU8OGMlKwFw4OJG23CmawF1_zKh?usp=sharing
